In [7]:
import re
from collections import defaultdict
from datetime import datetime

# Define log patterns for suspicious events
AUTH_FAILURE_PATTERN = r"authentication failure"
SESSION_OPEN_PATTERN = r"session opened"
FTP_CONNECTION_PATTERN = r"connection from"
IP_PATTERN = r"\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b"
DATE_PATTERN = r"\b\w{3} \d{2} \d{2}:\d{2}:\d{2}"

class LogAnalyzer:
    def __init__(self, log_data):
        self.log_data = log_data.splitlines()
        self.incidents = defaultdict(list)
        self.summary_report = defaultdict(int)

    def parse_log_entry(self, entry):
        """Extract date, IP, and message from each log entry."""
        date_match = re.search(DATE_PATTERN, entry)
        ip_match = re.search(IP_PATTERN, entry)
        date = datetime.strptime(date_match.group(), "%b %d %H:%M:%S") if date_match else None
        ip = ip_match.group() if ip_match else None
        return date, ip, entry.strip()

    def identify_suspicious_patterns(self):
        """Identify suspicious patterns and store incidents."""
        for entry in self.log_data:
            date, ip, message = self.parse_log_entry(entry)
            if re.search(AUTH_FAILURE_PATTERN, message, re.IGNORECASE):
                self.incidents['auth_failures'].append((date, ip, message))
                self.summary_report['auth_failures'] += 1
            elif re.search(SESSION_OPEN_PATTERN, message, re.IGNORECASE):
                self.incidents['session_opened'].append((date, ip, message))
                self.summary_report['session_opened'] += 1
            elif re.search(FTP_CONNECTION_PATTERN, message, re.IGNORECASE):
                self.incidents['ftp_connections'].append((date, ip, message))
                self.summary_report['ftp_connections'] += 1

    def generate_report(self):
        """Generate a summary report of incidents."""
        report = "Log Analysis Report\n"
        report += "=" * 20 + "\n"
        report += f"Total Authentication Failures: {self.summary_report['auth_failures']}\n"
        report += f"Total FTP Connections: {self.summary_report['ftp_connections']}\n"
        report += f"Total Session Opened: {self.summary_report['session_opened']}\n"
        report += "\nIncident Details:\n"

        for incident_type, entries in self.incidents.items():
            report += f"\n{incident_type.capitalize()}:\n"
            for date, ip, message in entries:
                report += f"Time: {date}, IP: {ip}, Message: {message}\n"
        return report

    def run_analysis(self):
        self.identify_suspicious_patterns()
        report = self.generate_report()
        print(report)

# Assign the multi-line log data to the log_data variable
log_data = """
Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 02:04:59 combo sshd(pam_unix)[20882]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20884]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20883]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20885]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20886]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20892]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20893]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20896]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20897]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20898]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 04:06:18 combo su(pam_unix)[21416]: session opened for user cyrus by (uid=0)
Jun 15 04:06:19 combo su(pam_unix)[21416]: session closed for user cyrus
Jun 15 04:06:20 combo logrotate: ALERT exited abnormally with [1]
Jun 15 04:12:42 combo su(pam_unix)[22644]: session opened for user news by (uid=0)
Jun 15 04:12:43 combo su(pam_unix)[22644]: session closed for user news
Jun 15 12:12:34 combo sshd(pam_unix)[23397]: check pass; user unknown
Jun 15 12:12:34 combo sshd(pam_unix)[23397]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:12:34 combo sshd(pam_unix)[23395]: check pass; user unknown
Jun 15 12:12:34 combo sshd(pam_unix)[23395]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:12:34 combo sshd(pam_unix)[23404]: check pass; user unknown
Jun 15 12:12:34 combo sshd(pam_unix)[23404]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:12:34 combo sshd(pam_unix)[23399]: check pass; user unknown
Jun 15 12:12:34 combo sshd(pam_unix)[23399]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:12:34 combo sshd(pam_unix)[23406]: check pass; user unknown
Jun 15 12:12:34 combo sshd(pam_unix)[23406]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:12:34 combo sshd(pam_unix)[23396]: check pass; user unknown
Jun 15 12:12:34 combo sshd(pam_unix)[23394]: check pass; user unknown
Jun 15 12:12:34 combo sshd(pam_unix)[23407]: check pass; user unknown
Jun 15 12:12:34 combo sshd(pam_unix)[23394]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:12:34 combo sshd(pam_unix)[23403]: check pass; user unknown
Jun 15 12:12:34 combo sshd(pam_unix)[23396]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:12:34 combo sshd(pam_unix)[23407]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:12:34 combo sshd(pam_unix)[23403]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:12:34 combo sshd(pam_unix)[23412]: check pass; user unknown
Jun 15 12:12:34 combo sshd(pam_unix)[23412]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:13:19 combo sshd(pam_unix)[23414]: check pass; user unknown
Jun 15 12:13:19 combo sshd(pam_unix)[23414]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 12:13:20 combo sshd(pam_unix)[23416]: check pass; user unknown
Jun 15 12:13:20 combo sshd(pam_unix)[23416]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 14:53:32 combo sshd(pam_unix)[23661]: check pass; user unknown
Jun 15 14:53:32 combo sshd(pam_unix)[23661]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=061092085098.ctinets.com 
Jun 15 14:53:32 combo sshd(pam_unix)[23663]: check pass; user unknown
Jun 15 14:53:32 combo sshd(pam_unix)[23663]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=061092085098.ctinets.com 
Jun 15 14:53:32 combo sshd(pam_unix)[23664]: check pass; user unknown
Jun 15 14:53:32 combo sshd(pam_unix)[23664]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=061092085098.ctinets.com 
Jun 15 14:53:33 combo sshd(pam_unix)[23665]: check pass; user unknown
Jun 15 14:53:33 combo sshd(pam_unix)[23665]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=061092085098.ctinets.com 
Jun 15 14:53:34 combo sshd(pam_unix)[23669]: check pass; user unknown
Jun 15 14:53:34 combo sshd(pam_unix)[23669]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=061092085098.ctinets.com 
Jun 15 14:53:35 combo sshd(pam_unix)[23671]: check pass; user unknown
Jun 15 14:53:35 combo sshd(pam_unix)[23671]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=061092085098.ctinets.com 
Jun 15 14:53:35 combo sshd(pam_unix)[23673]: check pass; user unknown
Jun 15 14:53:35 combo sshd(pam_unix)[23673]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=061092085098.ctinets.com 
Jun 15 14:53:35 combo sshd(pam_unix)[23674]: check pass; user unknown
Jun 15 14:53:35 combo sshd(pam_unix)[23674]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=061092085098.ctinets.com 
Jun 15 14:53:36 combo sshd(pam_unix)[23678]: check pass; user unknown
Jun 15 14:53:36 combo sshd(pam_unix)[23678]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=061092085098.ctinets.com 
Jun 15 14:53:36 combo sshd(pam_unix)[23677]: check pass; user unknown
Jun 15 14:53:36 combo sshd(pam_unix)[23677]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=061092085098.ctinets.com 
Jun 15 20:05:31 combo sshd(pam_unix)[24138]: check pass; user unknown
Jun 15 20:05:31 combo sshd(pam_unix)[24138]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=d211-116-254-214.rev.krline.net 
Jun 15 20:05:31 combo sshd(pam_unix)[24137]: check pass; user unknown
Jun 15 20:05:31 combo sshd(pam_unix)[24137]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=d211-116-254-214.rev.krline.net 
Jun 15 20:05:31 combo sshd(pam_unix)[24141]: check pass; user unknown
Jun 15 20:05:31 combo sshd(pam_unix)[24141]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=d211-116-254-214.rev.krline.net 
Jun 15 20:05:31 combo sshd(pam_unix)[24140]: check pass; user unknown
Jun 15 20:05:31 combo sshd(pam_unix)[24140]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=d211-116-254-214.rev.krline.net 
Jun 15 20:05:31 combo sshd(pam_unix)[24139]: check pass; user unknown
Jun 15 20:05:31 combo sshd(pam_unix)[24139]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=d211-116-254-214.rev.krline.net 
Jun 16 04:10:22 combo su(pam_unix)[25178]: session opened for user cyrus by (uid=0)
Jun 16 04:10:23 combo su(pam_unix)[25178]: session closed for user cyrus
Jun 16 04:10:24 combo logrotate: ALERT exited abnormally with [1]
Jun 16 04:16:17 combo su(pam_unix)[25548]: session opened for user news by (uid=0)
Jun 16 04:16:18 combo su(pam_unix)[25548]: session closed for user news
Jun 17 04:03:33 combo su(pam_unix)[27953]: session opened for user cyrus by (uid=0)
Jun 17 04:03:34 combo su(pam_unix)[27953]: session closed for user cyrus
Jun 17 04:03:36 combo logrotate: ALERT exited abnormally with [1]
Jun 17 04:09:43 combo su(pam_unix)[29190]: session opened for user news by (uid=0)
Jun 17 04:09:45 combo su(pam_unix)[29190]: session closed for user news
Jun 17 07:07:00 combo ftpd[29504]: connection from 24.54.76.216 (24-54-76-216.bflony.adelphia.net) at Fri Jun 17 07:07:00 2005 
Jun 17 07:07:00 combo ftpd[29508]: connection from 24.54.76.216 (24-54-76-216.bflony.adelphia.net) at Fri Jun 17 07:07:00 2005 
Jun 17 07:07:00 combo ftpd[29507]: connection from 24.54.76.216 (24-54-76-216.bflony.adelphia.net) at Fri Jun 17 07:07:00 2005 
Jun 17 07:07:00 combo ftpd[29505]: connection from 24.54.76.216 (24-54-76-216.bflony.adelphia.net) at Fri Jun 17 07:07:00 2005 
Jun 17 07:07:00 combo ftpd[29506]: connection from 24.54.76.216 (24-54-76-216.bflony.adelphia.net) at Fri Jun 17 07:07:00 2005 
Jun 17 07:07:00 combo ftpd[29509]: connection from 24.54.76.216 (24-54-76-216.bflony.adelphia.net) at Fri Jun 17 07:07:00 2005 
Jun 17 07:07:02 combo ftpd[29510]: connection from 24.54.76.216 (24-54-76-216.bflony.adelphia.net) at Fri Jun 17 07:07:02 2005 
Jun 17 07:07:04 combo ftpd[29511]: connection from 24.54.76.216 (24-54-76-216.bflony.adelphia.net) at Fri Jun 17 07:07:04 2005 
Jun 17 19:43:13 combo sshd(pam_unix)[30565]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=211.46.224.253  user=guest
Jun 17 20:29:26 combo sshd(pam_unix)[30631]: session opened for user test by (uid=509)
Jun 17 20:34:57 combo sshd(pam_unix)[30631]: session closed for user test
Jun 17 20:55:06 combo ftpd[30755]: connection from 82.252.162.81 (lns-vlq-45-tou-82-252-162-81.adsl.proxad.net) at Fri Jun 17 20:55:06 2005 
Jun 17 20:55:06 combo ftpd[30754]: connection from 82.252.162.81 (lns-vlq-45-tou-82-252-162-81.adsl.proxad.net) at Fri Jun 17 20:55:06 2005 
Jun 17 20:55:06 combo ftpd[30753]: connection from 82.252.162.81 (lns-vlq-45-tou-82-252-162-81.adsl.proxad.net) at Fri Jun 17 20:55:06 2005 
Jun 17 20:55:06 combo ftpd[30756]: connection from 82.252.162.81 (lns-vlq-45-tou-82-252-162-81.adsl.proxad.net) at Fri Jun 17 20:55:06 2005 
Jun 17 20:55:06 combo ftpd[30757]: connection from 82.252.162.81 (lns-vlq-45-tou-82-252-162-81.adsl.proxad.net) at Fri Jun 17 20:55:06 2005 
Jun 17 20:55:07 combo ftpd[30758]: connection from 82.252.162.81 (lns-vlq-45-tou-82-252-162-81.adsl.proxad.net) at Fri Jun 17 20:55:07 2005 
Jun 17 20:55:07 combo ftpd[30759]: connection from 82.252.162.81 (lns-vlq-45-tou-82-252-162-81.adsl.proxad.net) at Fri Jun 17 20:55:07 2005 
"""

# Run the log analyzer
log_analyzer = LogAnalyzer(log_data)
log_analyzer.run_analysis()

Log Analysis Report
Total Authentication Failures: 40
Total FTP Connections: 15
Total Session Opened: 7

Incident Details:

Auth_failures:
Time: 1900-06-14 15:16:01, IP: 218.188.2.4, Message: Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4
Time: 1900-06-14 15:16:02, IP: 218.188.2.4, Message: Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4
Time: 1900-06-15 02:04:59, IP: None, Message: Jun 15 02:04:59 combo sshd(pam_unix)[20882]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Time: 1900-06-15 02:04:59, IP: None, Message: Jun 15 02:04:59 combo sshd(pam_unix)[20884]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Time: 1900-06-15 02:04:59, IP: None, Message: Jun 15 02:04:59 combo sshd(pam_unix)[